In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

In [4]:


# --- CONFIGURAÇÕES ---
URL_BASE_CATEGORIA = "https://ilos.com.br/categoria/insights/"
TOTAL_PAGINAS = 78
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- FUNÇÃO 1: Extrai dados de UM artigo ---
def extrair_dados_artigo(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=20)
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, "html.parser")
        
        # 1. Título
        title_tag = soup.find("h1", class_="uicore-title")
        title = title_tag.get_text(strip=True) if title_tag else None

        # 2. Metadados (Autor, Data, Categoria)
        meta = soup.find("div", class_="uicore-entry-meta")
        author = None
        date = None
        category = None

        if meta:
            # Autor
            author_tag = meta.select_one('a[href*="/author/"]')
            if author_tag:
                author = author_tag.get_text(strip=True)
            
            # Data
            date_tag = meta.find("span", class_="ui-blog-date")
            if date_tag:
                date = date_tag.get_text(strip=True)
            
            # Categoria
            cat_tag = meta.select_one(".uicore-post-category a")
            if cat_tag:
                category = cat_tag.get_text(strip=True)

        # 3. Tags (via JSON-LD)
        tags = []
        for script in soup.find_all("script", type="application/ld+json"):
            try:
                data_ld = json.loads(script.string)
                graph = data_ld.get("@graph", [data_ld]) 
                for node in graph:
                    if node.get("@type") == "Article":
                        k = node.get("keywords")
                        if isinstance(k, str):
                            tags.extend([t.strip() for t in k.split(";")])
                        elif isinstance(k, list):
                            tags.extend(k)
            except:
                continue
        
        tags = list(set(tags))

        # 4. Conteúdo HTML
        article = soup.find("article")
        content_html = str(article) if article else None

        # 5. Thumbnail
        thumbnail = None
        og_image = soup.find("meta", property="og:image")
        if og_image:
            thumbnail = og_image["content"]

        return {
            "title": title,
            "author": author,
            "date": date,
            "thumbnail": thumbnail,
            "tags": tags,
            "category": category,
            "content_html": content_html,
            "source_url": url
        }

    except Exception as e:
        print(f"Erro ao extrair {url}: {e}")
        return None

# --- FUNÇÃO 2: Pega os links (Lógica Blindada) ---
def obter_links_da_pagina(numero_pagina):
    if numero_pagina == 1:
        url_pagina = URL_BASE_CATEGORIA
    else:
        url_pagina = f"{URL_BASE_CATEGORIA}page/{numero_pagina}/"

    print(f"\n--- Acessando lista: Página {numero_pagina} ---")
    
    try:
        response = requests.get(url_pagina, headers=HEADERS, timeout=20)
        soup = BeautifulSoup(response.text, "html.parser")
        
        links = []
        
        # Seleciona todos os blocos de artigo na listagem
        articles = soup.select("article.uicore-post")
        
        for article in articles:
            link_encontrado = None
            
            # CASO 1: Artigo com Thumbnail (Link é filho direto do wrapper)
            # O seletor > a garante que pega o link da imagem
            link_tag_img = article.select_one("div.uicore-post-wrapper > a")
            if link_tag_img:
                link_encontrado = link_tag_img.get('href')
            
            # CASO 2: Artigo SEM Thumbnail (Link dentro do info-wrapper)
            # Se falhou o caso 1, procuramos o Título e pegamos o link pai dele.
            # Isso funciona perfeitamente para links dentro de .uicore-post-info-wrapper
            if not link_encontrado:
                title_tag = article.select_one(".uicore-post-title")
                if title_tag:
                    parent_link = title_tag.find_parent("a")
                    if parent_link:
                        link_encontrado = parent_link.get('href')
            
            if link_encontrado:
                links.append(link_encontrado)
                
        return list(set(links)) 
        
    except Exception as e:
        print(f"Erro na paginação {numero_pagina}: {e}")
        return []

# --- EXECUÇÃO PRINCIPAL ---

todos_artigos = []

print("Iniciando scraping...")

for pagina in range(1, TOTAL_PAGINAS + 1):
    
    links = obter_links_da_pagina(pagina)
    print(f"> Encontrados {len(links)} artigos.")
    
    for link in links:
        print(f"   Extraindo: {link}")
        dados = extrair_dados_artigo(link)
        
        if dados:
            todos_artigos.append(dados)
        
        time.sleep(random.uniform(0.5, 1.5))
    
    # Salvamento parcial a cada 30 páginas
    if pagina % 30 == 0:
        print(f"--- Salvando progresso parcial (Página {pagina}) ---")
        with open("todos_artigos_ilos.json", "w", encoding="utf-8") as f:
            json.dump(todos_artigos, f, ensure_ascii=False, indent=2)
    
    time.sleep(1)

# --- SALVAMENTO FINAL ---
print("\n--- Finalizando e salvando arquivo completo ---")


print(f"Concluído! Total de {len(todos_artigos)} artigos salvos.")

Iniciando scraping...

--- Acessando lista: Página 1 ---
> Encontrados 12 artigos.
   Extraindo: https://ilos.com.br/emissoes-de-co2e-no-transporte-analise-do-cenario-global-e-brasileiro-e-implicacoes-para-a-logistica/
   Extraindo: https://ilos.com.br/a-logistica-dos-jogos-olimpicos-de-paris-2024/
   Extraindo: https://ilos.com.br/precisao-no-planejamento-o-calculo-dos-indicadores-para-a-eficiencia-operacional/
   Extraindo: https://ilos.com.br/bids-de-transportes-etapas-e-cuidados-para-se-tomar/
   Extraindo: https://ilos.com.br/estoque-rouba-a-cena-nos-custos-logisticos-do-brasil-em-2023/
   Extraindo: https://ilos.com.br/o-impacto-da-gestao-de-estoques-no-retorno-para-os-acionistas/
   Extraindo: https://ilos.com.br/a-logistica-e-o-supply-chain-nos-proximos-30-anos/
   Extraindo: https://ilos.com.br/csl-e-ifr-como-converter-o-nivel-de-servico/
   Extraindo: https://ilos.com.br/cobertura-de-estoque-importancia-e-aplicacao-pratica-do-indicador/
   Extraindo: https://ilos.com.br/movim

In [5]:
with open("todos_artigos_ilos.json", "w", encoding="utf-8") as f:
    json.dump(todos_artigos, f, ensure_ascii=False, indent=2)